## Imports

In [25]:
import sys
sys.path.append("..")

In [26]:
from snowflake.snowpark import Session
from snowflake.snowpark.functions import when, col, count
from datetime import date
from helpers import SnowflakeHelper
import json
import os

In [27]:
snowflake_helper = SnowflakeHelper()
snowflake_config = './../helpers/snowflake_config.json'
session = snowflake_helper.create_snowpark_session(snowflake_config)
session.use_schema("SAFEGUARDING_NYC_SCHEMA_SILVER")

[SUCCESS] Config file loaded successfully!
[SUCCESS] Snowspark Session created successfully!


## Extracting Data

In [28]:
use_of_force_incidents = session.table('use_of_force_incidents')
use_of_force_subjects = session.table('use_of_force_subjects')

In [29]:
use_of_force_incidents.show()

--------------------------------------------------------------------------------------------------
|"FORCETYPE"     |"INCIDENT PCT"  |"OCCURRENCE DATE"  |"TRI INCIDENT NUMBER"  |"PATROL BOROUGH"  |
--------------------------------------------------------------------------------------------------
|Physical Force  |75              |09/21/2023         |2023094962399          |Brooklyn          |
|Physical Force  |40              |09/13/2023         |2023059962344          |Bronx             |
|Physical Force  |45              |09/13/2023         |2023064962120          |Bronx             |
|Physical Force  |121             |09/08/2023         |2023140962087          |Staten Island     |
|Physical Force  |34              |08/27/2023         |2023053962127          |Manhattan         |
|Physical Force  |48              |08/24/2023         |2023067962183          |Bronx             |
|Physical Force  |42              |08/23/2023         |2023061962210          |Bronx             |
|Physical 

In [30]:
use_of_force_subjects.show()

-------------------------------------------------------------------------------------------
|"SUBJECT RACE"  |"FORCE AGAINST MOS"  |"TRI INCIDENT NUMBER"  |"SUBJECT GENDER"  |"AGE"  |
-------------------------------------------------------------------------------------------
|BLACK           |Physical Force       |2020020962067          |MALE              |NULL   |
|BLACK           |Physical Force       |2020020962143          |MALE              |NULL   |
|BLACK           |Physical Force       |2020024962080          |MALE              |NULL   |
|BLACK           |Physical Force       |2020025962038          |MALE              |NULL   |
|BLACK           |Physical Force       |2020025962049          |MALE              |NULL   |
|BLACK           |Physical Force       |2020025962140          |MALE              |NULL   |
|BLACK           |Physical Force       |2020026962081          |MALE              |NULL   |
|BLACK           |Physical Force       |2020032962033          |MALE            

## Joining Use Of Forces Tables

In [31]:
use_of_force_combined = use_of_force_incidents.join(
    use_of_force_subjects, 
    onColumns="TRI INCIDENT NUMBER", 
    how="inner"
)
use_of_force_combined.show()

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"FORCETYPE"        |"INCIDENT PCT"  |"OCCURRENCE DATE"  |"l_6k1v_TRI INCIDENT NUMBER"  |"PATROL BOROUGH"  |"SUBJECT RACE"  |"FORCE AGAINST MOS"  |"r_fuo3_TRI INCIDENT NUMBER"  |"SUBJECT GENDER"  |"AGE"  |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|Electrical Weapon  |67              |03/28/2022         |2022086962065                 |Brooklyn          |BLACK           |Physical Force       |2020020962067                 |MALE              |NULL   |
|Electrical Weapon  |1               |03/13/2022         |2022020962038                 |Manhattan         |BLACK           |Physical Force       |2020020962067                

In [32]:
use_of_force_combined = use_of_force_combined.drop(['r_fuo3_TRI INCIDENT NUMBER'])
use_of_force_combined = use_of_force_combined.withColumnRenamed("l_6k1v_TRI INCIDENT NUMBER", "TRI INCIDENT NUMBER")
use_of_force_combined.show()

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"FORCETYPE"        |"INCIDENT PCT"  |"OCCURRENCE DATE"  |"TRI INCIDENT NUMBER"  |"PATROL BOROUGH"  |"SUBJECT RACE"  |"FORCE AGAINST MOS"  |"SUBJECT GENDER"  |"AGE"  |
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------
|Electrical Weapon  |67              |03/28/2022         |2022086962065          |Brooklyn          |BLACK           |Physical Force       |MALE              |NULL   |
|Electrical Weapon  |1               |03/13/2022         |2022020962038          |Manhattan         |BLACK           |Physical Force       |MALE              |NULL   |
|Electrical Weapon  |106             |03/08/2022         |2022125962036          |Queens            |BLACK           |Physical Force       |MALE              |N

In [24]:
use_of_force_combined.count()

29723

## Exporting Data to Gold Medallion

In [23]:
table_name = "SAFEGUARDING_NYC_SCHEMA_GOLD.use_of_force_combined"
use_of_force_combined.write.saveAsTable(table_name, mode="overwrite")